In [1]:
import geopandas as gpd
from sqlalchemy import create_engine
from MyCreds.mycreds import Capstone_AWS_SG
from collections import defaultdict
import re

In [2]:
hdb_geo_query = """SELECT *
    FROM hdb_property_info a
    LEFT JOIN sg_buildings_postal_geo b
    ON CONCAT(a.blk_no,' ',a.street) = CONCAT(b.blk_no,' ',b.short_r_name)"""

# engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_PG.username}:{Capstone_AWS_PG.password}@{Capstone_AWS_PG.host}/capstone', echo=False)
engine = engine = create_engine(f'postgresql+psycopg2://{Capstone_AWS_SG.username}:{Capstone_AWS_SG.password}@{Capstone_AWS_SG.host}/Capstone', echo=False)

with engine.connect() as cnxn:
    df = gpd.read_postgis(hdb_geo_query, cnxn, geom_col='geometry')

df.columns = ['block_number', 'street', 'max_floor_lvl', 'year_completed', 'residential',
              'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark',
              'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units',
              '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold',
              'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental',
              '2room_rental', '3room_rental', 'other_room_rental', 'building_id',
              'address', 'blk_no', 'building', 'latitude', 'longitude', 'longtitude',
              'postal', 'road_name', 'short_r_name', 'searchval', 'x', 'y',
              'address_to_match', 'geometry']


df.dropna(inplace=True)
df

,block_number,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,longitude,longtitude,postal,road_name,short_r_name,searchval,x,y,address_to_match,geometry
0,100,ALJUNIED CRES,12,1977,Y,N,N,Y,N,N,...,103.884129,103.884129,380100,ALJUNIED CRESCENT,ALJUNIED CRES,100 ALJUNIED CRESCENT SINGAPORE 380100,33654.741815,33690.511066,100 ALJUNIED CRES,POINT (103.88413 1.32096)
1,100,ALJUNIED CRES,12,1977,Y,N,N,Y,N,N,...,103.884167,103.884167,380100,ALJUNIED CRESCENT,ALJUNIED CRES,BETHEL TOTS CENTRE,33658.975731,33691.305842,100 ALJUNIED CRES,POINT (103.88417 1.32097)
2,100,BEDOK NTH AVE 4,14,1978,Y,N,N,N,N,N,...,103.939033,103.939033,460100,BEDOK NORTH AVENUE 4,BEDOK NTH AVE 4,PEARL GARDEN,39764.880242,35172.068877,100 BEDOK NTH AVE 4,POINT (103.93903 1.33436)
3,100,C'WEALTH CRES,10,1964,Y,N,N,N,N,N,...,103.801272,103.801272,140100,COMMONWEALTH CRESCENT,C'WEALTH CRES,CRESCENT VIEW,24433.566857,32224.946100,100 C'WEALTH CRES,POINT (103.80127 1.30771)
4,100,LOR 1 TOA PAYOH,16,1968,Y,N,N,N,N,N,...,103.848586,103.848586,310100,LORONG 1 TOA PAYOH,LOR 1 TOA PAYOH,TOA PAYOH PALM SPRING,29699.068330,35747.703334,100 LOR 1 TOA PAYOH,POINT (103.84859 1.33956)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15210,9,ST. GEORGE'S RD,12,1984,Y,N,N,Y,N,N,...,103.862995,103.862995,320009,SAINT GEORGE'S ROAD,ST. GEORGE'S RD,SAINT GEORGE'S EAST GARDENS,31302.697707,33996.188644,9 ST. GEORGE'S RD,POINT (103.86300 1.32372)
15211,9,TECK WHYE LANE,13,1977,Y,Y,N,Y,N,N,...,103.754146,103.754146,680009,TECK WHYE LANE,TECK WHYE LANE,9 TECK WHYE LANE SINGAPORE 680009,19189.033682,40222.922280,9 TECK WHYE LANE,POINT (103.75415 1.38004)
15212,9,TELOK BLANGAH CRES,2,1974,N,Y,N,N,N,N,...,103.819409,103.819409,090009,TELOK BLANGAH CRESCENT,TELOK BLANGAH CRES,UOB TELOK BLANGAH CRESCENT,26451.926513,28874.857785,9 TELOK BLANGAH CRES,POINT (103.81941 1.27741)
15213,9,TELOK BLANGAH CRES,2,1974,N,Y,N,N,N,N,...,103.819408,103.819408,090009,TELOK BLANGAH CRESCENT,TELOK BLANGAH CRES,9 TELOK BLANGAH CRESCENT SINGAPORE 090009,26451.914890,28874.795433,9 TELOK BLANGAH CRES,POINT (103.81941 1.27741)


In [6]:
df.dtypes

block_number               object
street                     object
max_floor_lvl               int64
year_completed              int64
residential                object
commercial                 object
market_hawker              object
miscellaneous              object
multistorey_carpark        object
precinct_pavilion          object
bldg_contract_town         object
total_dwelling_units        int64
1room_sold                  int64
2room_sold                  int64
3room_sold                  int64
4room_sold                  int64
5room_sold                  int64
exec_sold                   int64
multigen_sold               int64
studio_apartment_sold       int64
1room_rental                int64
2room_rental                int64
3room_rental                int64
other_room_rental           int64
building_id               float64
address                    object
blk_no                     object
building                   object
latitude                  float64
longitude     

In [9]:
address_inv_index = defaultdict(list)
for i, address in enumerate((df['block_number'] + " " + df['street'] + " " + df['postal'])):
    address = " ".join(*re.findall('(^\d+)(...+)', address)).replace('  ', ' ')
    for word in address.split():
        address_inv_index[word].append(int(df.iloc[i]['building_id']))

In [10]:
address_inv_index

defaultdict(list,
            {'100': [44432, 44433, 66896, 10821, 36846, 37447],
             'ALJUNIED': [44432,
              44433,
              44434,
              44435,
              44436,
              44437,
              44439,
              44438,
              44440,
              44441,
              44442,
              44444,
              44443,
              44445,
              44446,
              44448,
              44449,
              44447,
              44450,
              44451,
              44455,
              44454,
              44453,
              44452,
              44456,
              44457,
              44458,
              44459,
              44461,
              44462,
              44460,
              44466,
              44463,
              44464,
              44465,
              44474,
              44475,
              42807,
              42806,
              42808,
              42809,
              42810,
              42811,
   